<a href="https://colab.research.google.com/github/farhadrgh/polyTopBuilder/blob/master/polyTopBuilder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import warnings
warnings.filterwarnings('ignore')

## Construct(), Coord(), Topol()

In [0]:
import sys, os
import numpy as np

# input parameters
Np = 10 # degree of polymerization Np
Npeg = 4 # degree of polymerization of PEG SC
resId = 1 # starting residue ID
atmId = 1 # starting atom ID
pType = "PEGMA"
Nsc = 3 if pType == "PCBMA" else Npeg +1


def header(pType,Np):
    return "; %s with %d monomers\n\n[ moleculetype ]\n; molname nrexcl \n%s    1\n" \
            % (pType,Np, pType+str(Np))
    
    
def printParam(mode,*args):
    
    if mode == "atoms":
        return "%5.d %5s %5.d %5s %5s %5.d %7.4f \n" %\
        (args[0],args[1],args[2],args[3],args[4],args[5],args[6])
        
    elif mode == "bonds":
        return "%5.d %5.d %6.d %10.4f %5.d %s\n" %\
        (args[0],args[1],args[2],args[3],args[4],args[5])
        
    elif mode == "angles":
        return "%5.d %5.d %5.d %6.d %5.d %5.d %s\n" %\
        (args[0],args[1],args[2],args[3],args[4],args[5],args[6])
    
    elif mode == "dihedrals":
        return "%5.d %5.d %5.d %5.d %6.d  %7.4f %7.4f %7.4f %7.4f %7.4f %7.4f %s\n" %\
        (args[0],args[1],args[2],args[3],args[4],args[5],args[6],args[7],args[8],args[9],args[10],args[11])
   

def Monomer(start, ID, pType, Nsc,s0=0):
    "Make monomer pType"

    atoms = ""
    bonds = ""
    angles= ""
    dihedrals = ""

    # parameters
    rbb = 0.289  # backbone bond
    kbb = 21100
    rsb = 0.282  # bacbone-sc
    ksb = 17000
    # Angles
    tbb  = 175   # backbone angle
    ktbb = 200 #13    
    tsb  = 90    # backbone-sc
    ktsb = 50 #13
    
    if pType == "PCBMA":   
        rsc  = 0.282
        ksc  = 17000
        tsc  = 144
        ktsc = 67
        
    elif pType == "PEGMA":
        rsc  = 0.322
        ksc = 7000
        tsc  = 122
        ktsc = 400
    
    # [ atoms ]
    mode = "atoms"
    if pType == "PCBMA":
        atoms  = printParam(mode, start ,"SC1",ID,pType[:3],"BB",start,0.0000)    
        atoms += printParam(mode, start+1,"Na",ID,pType[:3],"S1",start+1,0.0000)  
        atoms += printParam(mode, start+2,"Qd",ID,pType[:3],"S2",start+2,1.0000)  
        atoms += printParam(mode, start+3,"Qa",ID,pType[:3],"S3",start+3,-1.0000) 

    elif pType == "PEGMA":
        atoms  = printParam(mode, start ,"SC1",ID,pType[:3],"BB",start,0.0000)    
        atoms += printParam(mode, start+1,"Na",ID,pType[:3],"S1",start+1,0.0000)  
        for i in range(Nsc-1):
            atoms += printParam(mode, start+2+i,"SP0",ID,pType[:3],"S"+str(i+2),start+2+i,0.0000)
       
    
    # [ bonds ]
    mode = "bonds"

    # backbone
    if start == 2+s0: 
        bonds  = printParam(mode,1+s0,start,1,rbb,kbb,"; BB")
        bonds += printParam(mode,start,start+(Nsc+1),1,rbb,kbb,"; BB")
    elif start < Np*(Nsc+1)+1+s0:
        bonds += printParam(mode,start,start+(Nsc+1),1,rbb,kbb,"; BB")
                
    # side chain
    bonds += printParam(mode,start,start+1,1,rsb,ksb,"; SB")
    for i in range(1,Nsc):
        bonds += printParam(mode,start+i,start+i+1,1,rsc,ksc,"; SC") 
        
                
    # [ angles ]
    mode = "angles"
    
    # backbone
    if start == 2+s0:
        angles = printParam(mode,1+s0, start, start+(Nsc+1),2,tbb,ktbb,"; BB")
        angles+= printParam(mode,1+s0,start,start+1,2,tsb,ktsb,"; SCBB")  

    # side chain
    for j in range(Nsc-1):
        angles+= printParam(mode,start+j,start+(j+1),start+(j+2),2,tsc,ktsc,"; SC") 

    # bb/sc
    if start < (Np-1)*(Nsc+1)+1+s0:
        angles+= printParam(mode,start,start+(Nsc+1),start+1+(Nsc+1),2,tsb,ktsb,"; SCBB") 
        angles+= printParam(mode,start,start+(Nsc+1),start+2*(Nsc+1),2,tbb,ktbb,"; BB") 

    
    # [ dihedrals ]        
    if  pType == "PEGMA":       
        mode = "dihedrals"
        if start > 1+s0:
            dihedrals = printParam(mode,start+2,start+3,start+4,start+5,3,-2.73235989, 4.37322998, -0.11301000, -3.63837004, 0.90266800, 1.14602995,"; PEG") 
            
            
    return {'atoms': atoms, 'bonds': bonds, 'angles': angles, 'dihedrals' : dihedrals}


def construct_old(pType,Np,Nsc):
    "Make polymer pType"
    
    fout = open("%s%d.itp" % (pType,Np), "w")
    fout.write(header(pType=pType,Np=Np))
    fout.write("[ atoms ]\n;  nr  type  resnr  residu  atom  cgnr  charge\n")
    for mon in range(1,Np+1):
        start = (mon-1)*(Nsc+1) + 2

        if   mon == 1:    # first monomer
            fout.write(printParam("atoms" ,1 ,"SC1",1,pType[:3],"BB",1,0.0000))
            fout.write(Monomer(start, mon, pType=pType, Nsc=Nsc)['atoms'])

        elif mon == Np: # last monomer
            fout.write(Monomer(start, mon, pType=pType, Nsc=Nsc)['atoms'])
            fout.write(printParam("atoms",start+(Nsc+1),"SC1",mon,pType[:3],"BB",start+(Nsc+1),0.0000))

        elif mon < Np:  # midle monomers
            fout.write(Monomer(start, mon, pType=pType, Nsc=Nsc)['atoms'])

    fout.write("[ bonds ]\n")
    for mon in range(1,Np+1):
        start = (mon-1)*(Nsc+1) + 2
        fout.write(Monomer(start, mon, pType=pType, Nsc=Nsc)['bonds'])

    fout.write("[ angles ]\n")
    for mon in range(1,Np+1):
        start = (mon-1)*(Nsc+1) + 2
        fout.write(Monomer(start, mon, pType=pType, Nsc=Nsc)['angles'])

    if pType=="PEGMA":
        fout.write("[ dihedrals ]\n")
        for mon in range(1,Np+1):
            start = (mon-1)*(Nsc+1) + 2
            fout.write(Monomer(start, mon, pType=pType, Nsc=Nsc)['dihedrals'])
            
def construct(pType,Np,Nsc,site=1,s0=0,res0=0, segname=""):
    """Make polymer pType, Np monomers, Nsc including Na, site on prot, 
       s0+1 begining of poly atom, res0+1 begining of poly res"""
     
    if s0 == 0 :
        fout = open("%s%d.itp" % (pType,Np), "w")
        fout.write(header(pType=pType,Np=Np))
        
    else:
        fout = open("out%s.itp" % segname, "a")
        fout.write("\n; %s%d conj to %d atm\n" % (pType,Np,site))
        
    fout.write("[ atoms ]\n;  nr  type  resnr  residu  atom  cgnr  charge\n")
    for mon in range(1,Np+1):
        start = (mon-1)*(Nsc+1) + 2

        if   mon == 1:    # first monomer
            fout.write(printParam("atoms" ,1+s0 ,"SC1",1+res0,pType[:3],"BB",1+s0,0.0000))
            fout.write(Monomer(start+s0, mon+res0, pType, Nsc, s0)['atoms'])

        elif mon == Np: # last monomer
            fout.write(Monomer(start+s0, mon+res0, pType, Nsc, s0)['atoms'])
            fout.write(printParam("atoms",start+(Nsc+1)+s0,"SC1",mon+res0,pType[:3],"BB",start+(Nsc+1)+s0,0.0000))

        elif mon < Np:  # midle monomers
            fout.write(Monomer(start+s0, mon+res0, pType, Nsc, s0)['atoms'])

    fout.write("[ bonds ]\n")
    if s0 != 0: 
        rbb = 0.289  # backbone bond
        kbb = 21100
        fout.write(printParam("bonds",site,s0+1,1,rbb,kbb,"; CONJ"))
        
    for mon in range(1,Np+1):
        start = (mon-1)*(Nsc+1) + 2
        fout.write(Monomer(start+s0, mon+res0, pType, Nsc, s0)['bonds'])

    fout.write("[ angles ]\n")
    for mon in range(1,Np+1):
        start = (mon-1)*(Nsc+1) + 2
        fout.write(Monomer(start+s0, mon+res0, pType, Nsc, s0)['angles'])

    if pType=="PEGMA":
        fout.write("[ dihedrals ]\n")
        for mon in range(1,Np+1):
            start = (mon-1)*(Nsc+1) + 2
            fout.write(Monomer(start+s0, mon+res0, pType, Nsc, s0)['dihedrals'])

    fout.close() 

def coord(pType,Np,Nsc):
    
    fout = open("%s%d.gro" % (pType,Np), "w")
    fout.write("%s%d by polyTopGen\n" % (pType,Np))
    fout.write("%d\n" % (Np*(Nsc+1)+2))
    fout.write("%8s%7s%5.d%8.3f%8.3f%8.3f\n" % (str(1)+pType[:3],"BB",1,0,0,0))
    
    for i in range(Np):
        start = i*(Nsc+1)+2
        fout.write("%8s%7s%5.d%8.3f%8.3f%8.3f\n" % (str(i+1)+pType[:3],"BB",start,0.32*(1+i),0,0))
        for j in range(Nsc):
            fout.write("%8s%7s%5.d%8.3f%8.3f%8.3f\n" % (str(i+1)+pType[:3],"S"+str(j+1),start+(j+1),0.32*(1+i),((-1)**i)*0.32*(1+j),0))
            
    fout.write("%8s%7s%5.d%8.3f%8.3f%8.3f\n" % (str(i+1)+pType[:3],"BB",start+Nsc+1,0.32*(1+Np),0,0))
    fout.write("%10.5f%10.5f%10.5f" % (0,0,0))

    
def vtf_builder(pType=pType,Np=Np,Nsc=Nsc,water=False):

    names = ["BB", "S1", "S2", "S3","S4","S5"]

    outname = "conf-nw.vtf"
    if water:
        outname = "conf.vtf"
    
    fout = open(outname, "w")
    fout.write("atom 0 name BB resname PCB\n")# SC1"
    fout.write("atom 1 name BB resname PCB\n")# SC1"

    for i in range(2,Np*(Nsc+1)+2):
        fout.write("atom %d name %s resname %s\n" % (i,names[(i-1)%(Nsc+1)],pType[:3]))#,resnames[i%4-1])
        #print (i-1)%(Nsc+1)
    if water:
        for w in range(water):
            fout.write("atom %d name %s resname %s\n" % (i+w+1,"W", "WAT"))

    fout.write("bond 0:1\n")
    for i in range(Np):
        for j in range(1,Nsc+1):
            fout.write("bond %d:%d\n" % (i*(Nsc+1)+j,i*(Nsc+1)+j+1))
            if j == 1:
                fout.write("bond %d:%d\n" % (i*(Nsc+1)+1, (i+1)*(Nsc+1)+1))


In [0]:
poly = "PEGMA"
Np = 10
Nsc = 4 # including Na

coord(poly,Np,Nsc)
data = open(poly+str(Np)+".gro","r").read()

construct(poly,Np,Nsc) # Nsc including Na
data = open(poly+str(Np)+".itp","r").read()

#vtf_builder(poly,Np,Nsc)
#data = open("conf-nw.vtf","r").read()

#print data

## Visualize()

In [0]:
def vis(top,coord):
    
    import nglview as nv
    from nglview.datafiles import GRO
    import MDAnalysis as md
    
    u = md.Universe(top,coord)
    return nv.show_mdanalysis(u)
    
    
top = '../181017-PEGMA10/01-EM/topol.tpr'
xtc = '../181017-PEGMA10/01-EM/traj.trr'
gro = 'PEGMA10.gro'

vis(gro,xtc)

NGLWidget(count=522)

## Conjugate()

In [0]:
import numpy as np
import MDAnalysis as md
from MDAnalysis.analysis import align


def rotation_matrix(A,B):
    "rotate all the components to vec B"
    ax = A[0]
    ay = A[1]
    az = A[2]

    bx = B[0]
    by = B[1]
    bz = B[2]

    au = A/(np.sqrt(ax*ax + ay*ay + az*az))
    bu = B/(np.sqrt(bx*bx + by*by + bz*bz))

    R=np.array([[bu[0]*au[0], bu[0]*au[1], bu[0]*au[2]],
               [bu[1]*au[0], bu[1]*au[1], bu[1]*au[2]], 
               [bu[2]*au[0], bu[2]*au[1], bu[2]*au[2]]])
    
    return(R)


def norm(A):
    
    s = 0
    for i in A:
        s += i**2
    return np.sqrt(s)


def angle(A,B):
    
    ax = A[0]
    ay = A[1]
    az = A[2]

    bx = B[0]
    by = B[1]
    bz = B[2]
    
    sin = norm(np.cross(A,B))
    cos = ax*bx + ay*by + az*bz
    rotangle = np.rad2deg(np.arctan(sin/cos))
    if rotangle < 0: rotangle = 180 - np.abs(rotangle)
        
    return rotangle 


def Conjugate(pdb,pType,Np,CTLYS):
    
    upro   = md.Universe(pdb)
    pro    = md.AtomGroup(upro.select_atoms('all'))
    
    segdic = {}
    for segid in np.unique(pro.segids):
        segdic[segid] = md.AtomGroup(upro.select_atoms('segid %s' % segid))
        
    for resid in CTLYS:
        sidech = "SC1" if resid == 1 else  "SC2"
        site   = pro.select_atoms("resid %d and name %s" % (resid,sidech))        
    
        upol = md.Universe('%s%d.gro' % (pType,Np))
        pol  = md.AtomGroup(upol.select_atoms('all'))    

        proCOM = pro.center_of_geometry()
        sitCOM = site.center_of_geometry()

        proVec = sitCOM - proCOM
        polVec = pol.positions[-1]

        rotaxis  = np.cross(polVec,proVec) # axis of rotation
        rotangle = angle(proVec,polVec)    # angle of rotation

        pol.atoms.rotateby(rotangle,rotaxis,point=[0,0,0])
        pol.atoms.translate(site.positions[-1]+0.2*proVec)

        segname= site.segids[0]
        Uconj  = md.Merge(segdic[segname],pol) # returns Universe not AtomGroup
        segdic[segname] = md.AtomGroup(Uconj.select_atoms('all'))

        print "%s%d\t conjugated with %s" % (site.resnames[0],resid, pType+str(Np))
  
    # Merging conjugated sections 
    # md.Merge(segdic['A'],segdic['B'],segdic['C'])
    # merge every other value (if more than one)
    # to the first segment in dict
    # if protein has one section, last Uconj 
    if len(segdic.keys())>1:
        sortedKeys = sorted(segdic.keys()) # [A,B,C...]
        for key in sortedKeys[1:]:
            Utemp = md.Merge(segdic[sortedKeys[0]],segdic[key])
            segdic[sortedKeys[0]] = md.AtomGroup(Utemp.select_atoms('all'))
        conj = Utemp
    else:
        conj = Uconj
    
    # Write out the Merged Universe
    conj.atoms.write(filename="out.gro") # write Universe into file
    

import os 
if os.path.isfile('out.gro'): os.remove("out.gro")
CTLYS = [1, 36, 79, 84, 87, 90, 93, 169, 170, 175, 177, 202]
pType = 'PCBMA'
Np    = 10
pdb   = 'CG4cha.pdb'
Conjugate(pdb,pType,Np,CTLYS)

import nglview as nv
from nglview.datafiles import GRO
import MDAnalysis as md

gro = 'out.gro'
#gro = 'CG4cha.pdb'
u = md.Universe(gro)
nv.show_mdanalysis(u)

CYS1	 conjugated with PCBMA10
LYS36	 conjugated with PCBMA10
LYS79	 conjugated with PCBMA10
LYS84	 conjugated with PCBMA10
LYS87	 conjugated with PCBMA10
LYS90	 conjugated with PCBMA10
LYS93	 conjugated with PCBMA10
LYS169	 conjugated with PCBMA10
LYS170	 conjugated with PCBMA10
LYS175	 conjugated with PCBMA10
LYS177	 conjugated with PCBMA10
LYS202	 conjugated with PCBMA10


NGLWidget()

In [0]:
import os
for i in ['A','B','C']:
    if os.path.isfile('out%s.itp' % i): 
        os.remove("out%s.itp" % i)

def SegMinMax(universe):

    sel1 = universe.select_atoms("all")
    sel2 = universe.select_atoms("name BB")
    segids = np.unique(sel1.segids)
    MinMaxdic = dict(zip(segids,np.zeros((len(segids),4)))) # atmax resmax dict of each segment

    # find atom/res ID max of each segment
    for key,val in MinMaxdic.items():
        atm = np.where(sel1.segids==key) # array of indices of atoms in seg C
        res = np.where(sel2.segids==key)
        
        atmax = np.max(atm)+1
        atmin = np.min(atm)+1
        resmax= np.max(res)+1
        resmin= np.min(res)+1
        
        MinMaxdic[key] = [atmin,atmax,resmin,resmax]
        
    return MinMaxdic

poly = "PEGMA"
Np = 10
Nsc = 4 # including Na
poleng = 2+Np*(Nsc+1)

pdb   = 'CG4cha.pdb'
upro  = md.Universe(pdb)
pro   = upro.select_atoms("all")

nRes  = len(upro.residues)
nAtm  = len(upro.atoms)

segids = np.unique(pro.segids)  # protein segments (different itps)
segdic = dict(zip(segids,np.zeros(len(segids)))) # zero dict of each se
MinMaxdic = SegMinMax(upro)

CTLYS = [1, 36, 79, 84, 87, 90, 93, 169, 170, 175, 177, 202]

for resid in CTLYS:
    
    sidech = "SC1" if resid == 1 else  "SC2"
    sel    = upro.select_atoms("resid %d and name %s" % (resid,sidech))
    
    atmid  = sel.atoms.ids[0]     # atomid (in pdb) to be bonded to poly
    segname= sel.segids[0]
    
    atmin  = MinMaxdic[segname][0]
    atmax  = MinMaxdic[segname][1]
    resmin = MinMaxdic[segname][2]
    resmax = MinMaxdic[segname][3]

    site   = atmid-atmin+1       # atomid (in itp) to be bonded to poly
    s0     = int(poleng*segdic[segname]) + atmax - atmin + 1 # starting atmid of poly -1
    res0   = resmax - resmin + 1

    construct(poly,Np,Nsc,site=site,s0=s0,res0=res0,segname=segname) 
    print "Seg %s %s%d\t conjugated with %s" % (segname,sel.resnames[0],resid, pType+str(Np))
    segdic[segname] += 1

Seg A CYS1	 conjugated with PCBMA10
Seg B LYS36	 conjugated with PCBMA10
Seg B LYS79	 conjugated with PCBMA10
Seg B LYS84	 conjugated with PCBMA10
Seg B LYS87	 conjugated with PCBMA10
Seg B LYS90	 conjugated with PCBMA10
Seg B LYS93	 conjugated with PCBMA10
Seg C LYS169	 conjugated with PCBMA10
Seg C LYS170	 conjugated with PCBMA10
Seg C LYS175	 conjugated with PCBMA10
Seg C LYS177	 conjugated with PCBMA10
Seg C LYS202	 conjugated with PCBMA10


In [0]:
for resid in CTLYS:
    
    sidech    = "SC1" if resid == 1 else  "SC2"
    sel       = upro.select_atoms("resid %d and name %s" % (resid,sidech))
    segname   = sel.segids[0]
    sortedKeys = sorted(segdic.keys())
    
    if segname == sortedKeys[0]:
        print resid, resid
    else:
        resmin = MinMaxdic[segname][2]
        index  = sortedKeys.index(segname)
        print resid, resid-resmin-int(segdic[sortedKeys[index-1]])+1  


1 1
36 24
79 67
84 72
87 75
90 78
93 81
169 21
170 22
175 27
177 29
202 54


In [0]:
for n in segdic.keys():
    data = open("out%s.itp" % segname,"r").read()
    print data


; PEGMA10 conj to 42 atm
[ atoms ]
;  nr  type  resnr  residu  atom  cgnr  charge
  199   SC1    98   PEG    BB   199  0.0000 
  200   SC1    98   PEG    BB   200  0.0000 
  201    Na    98   PEG    S1   201  0.0000 
  202   SP0    98   PEG    S2   202  0.0000 
  203   SP0    98   PEG    S3   203  0.0000 
  204   SP0    98   PEG    S4   204  0.0000 
  205   SC1    99   PEG    BB   205  0.0000 
  206    Na    99   PEG    S1   206  0.0000 
  207   SP0    99   PEG    S2   207  0.0000 
  208   SP0    99   PEG    S3   208  0.0000 
  209   SP0    99   PEG    S4   209  0.0000 
  210   SC1   100   PEG    BB   210  0.0000 
  211    Na   100   PEG    S1   211  0.0000 
  212   SP0   100   PEG    S2   212  0.0000 
  213   SP0   100   PEG    S3   213  0.0000 
  214   SP0   100   PEG    S4   214  0.0000 
  215   SC1   101   PEG    BB   215  0.0000 
  216    Na   101   PEG    S1   216  0.0000 
  217   SP0   101   PEG    S2   217  0.0000 
  218   SP0   101   PEG    S3   218  0.0000 
  219   SP0   101

In [0]:
gro = "../CT/01-EM/conf.gro"
top = "../CT/01-EM/topol.tpr"
xtc = "../CT/04-NPTPR/trajout.xtc"
u = md.Universe(top,xtc)
nv.show_mdanalysis(u)

NGLWidget(count=251)